In [1]:
from mxnet import np, npx, autograd
from mxnet.gluon import Trainer
from mxnet.gluon.loss import SoftmaxCrossEntropyLoss
import mxnet.gluon.nn as nn
from mxnet import init
from d2l import mxnet as d2l
from pdb import set_trace
npx.set_np()

## Load Data

In [2]:
num_inputs = 28*28
num_classes = 10
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

In [3]:
def resized_iter(orig_iter, test=True):
    num_iterations = 0
    for X, y in orig_iter:
        if num_iterations >= 5 and test:
            break
        X = np.reshape(X, (X.shape[0], -1))
        yield X, y
        num_iterations += 1

## Define Model

In [4]:
net = nn.Sequential()
net.add(nn.Dense(num_classes))

### Initialize Params

In [5]:
net.initialize(init.Normal(sigma=0.01))

## Define Loss

In [6]:
def ce_loss(yhat, y):
    loss = SoftmaxCrossEntropyLoss(axis=1, from_logits=False)
    return loss(yhat, y[:, np.newaxis]).mean()

## Define Optimization Loop

In [7]:
lr = 0.1
num_epochs = 10
optim = Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})
for epoch in range(num_epochs):
    for X, y in resized_iter(train_iter, False):
        with autograd.record():
            yhat = net(X)
            loss = ce_loss(yhat, y)
        loss.backward()
        optim.step(batch_size=1)
    print(f'Epoch: {epoch}, Loss: {loss.item()}')

Epoch: 0, Loss: 0.864424467086792
Epoch: 1, Loss: 0.8582344651222229
Epoch: 2, Loss: 0.6851279735565186
Epoch: 3, Loss: 0.6579068303108215
Epoch: 4, Loss: 0.6317727565765381
Epoch: 5, Loss: 0.5884999632835388
Epoch: 6, Loss: 0.6406451463699341
Epoch: 7, Loss: 0.41396665573120117
Epoch: 8, Loss: 0.709810733795166
Epoch: 9, Loss: 0.5485646724700928


## Calculate accuracy

In [16]:
correct = 0
total = 0
for X, y in resized_iter(test_iter, False):
    preds = np.argmax(net(X), axis=1)
    correct += (preds.astype('int32') == y).sum()
    total += X.shape[0]

In [17]:
correct/total

array(0.8074)